<a href="https://colab.research.google.com/github/dfridland/NLP/blob/HW15/NLP_HW_15_products.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [ ]:
#!ls ../2

In [ ]:
assert True

#Small preprocess of the answers

question = None
written = False

with open("./TelegramBot/Constance.py.txt", "w") as fout:
    with open("./TelegramBot/Responses.py", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_33158/882813330.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [ ]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

with open("./TelegramBot/Responses.py", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_33158/2311395522.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [ ]:
#!pip install fasttext

In [ ]:
sentences = [i for i in sentences if len(i) > 2]
print('start train fasttext')
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5, workers=8)
modelFT.save("ft_model")

start train fasttext


In [ ]:
modelFT = FastText.load("ft_model")

In [ ]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(100 ,'angular')

index_map = {}
counter = 0

with open("./TelegramBot/Responses.py", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")

        index_map[counter] = spls[0]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_33158/2613819939.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

In [ ]:
modelFT = FastText.load("ft_model")

In [ ]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann')

True

In [ ]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[0, 1]

In [ ]:
shop_data = pd.read_csv("ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [ ]:
shop_data.shape

(35548, 8)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [ ]:
!ls

Echobotv20.ipynb
Otvety.txt
ProductsDataset.csv
TelegramBot
Telegram_bot_tutorial.ipynb
Untitled.ipynb
Untitled1.ipynb
chat-bot_imgs.ipynb
chat_bot.ipynb
ft_model
ft_model.wv.vectors_ngrams.npy
gbnlp-afdu-aac18209b96f (1).json
hw15.txt
hw16.txt
index_shop.pkl
index_speaker.pkl
lesson_15.ipynb
lesson_15_2.ipynb
prepared_answers.txt
shop.ann
speaker.ann
Пример_разбора_ДЗ_NLP_14_1 (1).ipynb
Пример_разбора_ДЗ_NLP_14_3.ipynb


In [ ]:
import os
filepath = './index_speaker.pkl'
if os.path.exists(filepath):
    file = open('./index_speaker.pkl', 'rb') as f
    codedata = pickle.load(f)
    file.close()
else:
    print("File not present at desired location")

In [ ]:
with open('./index_speaker.pkl', 'rb') as f:
    index_map = pickle.load(f)

In [ ]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))

In [ ]:
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]

In [ ]:
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [ ]:
negative_texts[3]

'usermessage strinputtextlower'

In [ ]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

In [ ]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [ ]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9995782370307887

In [ ]:
import pickle

In [ ]:
# assert
import pickle

with open("index_speaker.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [ ]:
np.mean(tfidf_vect.idf_)

10.16859539453302

In [ ]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'пальто': 9.646500675887538,
 'женский': 7.7295780637054765,
 'продать': 10.562791407761692,
 'состояние': 10.562791407761692,
 'хороший': 10.562791407761692,
 'наличие': 10.562791407761692,
 'разный': 10.562791407761692,
 'вещь': 10.562791407761692,
 'размер': 10.562791407761692,
 '5456': 10.562791407761692,
 'сапог': 10.562791407761692,
 'кожаныеосень': 10.562791407761692,
 'состоянииить': 10.562791407761692,
 '3738': 10.562791407761692,
 'набор': 8.953353495327592,
 'стильно': 10.562791407761692,
 'чемодан': 10.562791407761692,
 'раскладной': 10.562791407761692,
 'произвый': 9.464179119093583,
 'корея': 10.562791407761692,
 'разм': 10.562791407761692,
 'аысота': 10.157326299653528,
 '60': 10.562791407761692,
 'смотреть': 10.562791407761692,
 'ширина': 10.562791407761692,
 '40': 10.562791407761692,
 'глубина': 10.562791407761692,
 '23': 9.869644227201746,
 'смв': 10.562791407761692,
 'разложа': 10.562791407761692,
 'вид': 10.562791407761692,
 '31': 10.562791407761692,
 'легко': 8.95

In [ ]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

/var/folders/g_/9v3q8zfn4dj84j48wgjpsd7c0000gn/T/ipykernel_33158/3283345512.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


  0%|          | 0/35548 [00:00<?, ?it/s]

True

In [ ]:
ft_index_shop = annoy.AnnoyIndex(100, 'angular')
ft_index_shop.load('shop.ann')

midf = np.mean(tfidf_vect.idf_)

In [ ]:
assert True
import pickle

with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [ ]:
'бегать' in modelFT.wv

True

In [ ]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word] * 1 # idfs.get(word, midf)
            n_ft += 1 # idfs.get(word, midf)
    return vector_ft / n_ft

In [ ]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 5, include_distances=True)

([315, 730, 761, 2624, 2681],
 [1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608,
  1.4142135381698608])

In [ ]:
import os
import logging
from telegram import Update
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext

In [ ]:
updater = Updater("6094279951:AAH19NaLLeiCO0CfIBwFSKOXUOGkAQAkQo0", use_context=True)  # Токен API к Telegram
dispatcher = updater.dispatcher

In [ ]:
def startCommand(update: Update, context: CallbackContext):
    update.message.reply_text('Добрый день!')

def textMessage(update: Update, context: CallbackContext):
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)

    if prediction[0] == 1:
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            update.message.reply_text("title: {} image: {}".format(title, image))
            # bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    if distances[0] > 0.35:
        print(distances[0])
        update.message.reply_text("Моя твоя не понимать")
        return
    update.message.reply_text(index_map[ft_index_val[0]])

In [ ]:
# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler("start", startCommand))
dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, textMessage))

# Start the Bot
updater.start_polling()
updater.idle()